In [2]:
import pandas

In [3]:
import ee
ee.Initialize()

In [4]:
inputCSV = ee.FeatureCollection('users/ccrs2fy2020/transformedFeatures')
inputCSV = inputCSV.toList(count=50000)

In [5]:
def extractBands(feature):
    feature = ee.Feature(feature)
    return feature.toArray(properties=['B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                          'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                          'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                          'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                          'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                          'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                          'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI']).toList()

In [6]:
def extractVI(feature):
    feature = ee.Feature(feature)
    return feature.toArray(properties=['FCOVER']).toList()

In [7]:
inputList = inputCSV.map(extractBands)
outputList = inputCSV.map(extractVI)

In [8]:
X = ee.Array(inputList)
y = ee.Array(outputList)

n = X.length().get([0])
m = X.length().get([1])

In [9]:
def centre(output):
    output = ee.Array(output)
    mean = output.reduce(ee.Reducer.mean(), [0]).get([0,0])
    return output.subtract(mean)

In [10]:
def normalize(inputs):
    inputs = ee.Array(inputs)

    inputMeans = inputs.reduce(ee.Reducer.mean(), [0])
    inputMeans = inputMeans.repeat(0, n)
    inputs = inputs.subtract(inputMeans)
    inputs = inputs.pow(2).reduce(ee.Reducer.sum(), [0]).pow(-0.5).repeat(0, n).multiply(inputs)

    return inputs

In [11]:
def standardize(inputs):
    inputs = ee.Array(inputs)

    inputMeans = inputs.reduce(ee.Reducer.mean(), [0])
    inputMeans = inputMeans.repeat(0, n)
    inputs = inputs.subtract(inputMeans)

    inputSTDevs = inputs.reduce(ee.Reducer.stdDev(), [0])
    inputSTDevs = inputSTDevs.repeat(0, n)
    inputs = inputs.divide(inputSTDevs)

    return inputs

In [12]:
def setZero(num):
    num = ee.Number(num);
    return ee.Algorithms.If(num.abs().lt(0.00001), [0], [num])

In [13]:
X = normalize(X)
y = centre(y)

In [14]:
def LARSregression(iteration, inputs):
    inputs = ee.Dictionary(inputs)
    prediction = inputs.getArray('prediction')
    coeff_arr = inputs.getArray('coeff_arr')

    c = X.matrixTranspose().matrixMultiply(y.subtract(prediction))
    c_abs = c.abs()
    C_max = c_abs.get(c_abs.argmax())

    maxLocs = c_abs.gte(C_max.subtract(0.00001))
    signs = c.divide(c_abs)

    signs_j = maxLocs.multiply(signs).matrixTranspose()
    signs_jc = signs_j.abs().subtract(1).multiply(-1)

    A = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_j).toList().get(0))
    A_c = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_jc).toList().get(0))

    signMatrix_j = signs_j.repeat(0, n)

    X_A = X.multiply(signMatrix_j).mask(signs_j)
    j = X_A.length().get([1])

    G_A = X_A.matrixTranspose().matrixMultiply(X_A)

    V1_A = ee.Array(ee.List.repeat([1], j))

    G_Ai = G_A.matrixInverse()

    A_A = V1_A.matrixTranspose().matrixMultiply(G_Ai).matrixMultiply(V1_A).get([0,0]).pow(-0.5)

    w_A = G_Ai.matrixMultiply(V1_A).multiply(A_A)

    u_A = X_A.matrixMultiply(w_A)

    a = X.matrixTranspose().matrixMultiply(u_A)


    def computeGammaRRay(index_j):
        minus_j = C_max.subtract(c.get([index_j, 0])).divide(A_A.subtract(a.get([index_j, 0])))
        plus_j = C_max.add(c.get([index_j, 0])).divide(A_A.add(a.get([index_j, 0])))

        gammaRRay = ee.Array([minus_j, plus_j])
        gammaRRay = gammaRRay.mask(gammaRRay.gte(0))
        gammaRRay = gammaRRay.multiply(-1)

        return gammaRRay.get(gammaRRay.argmax())


    gammaRRay = ee.Array([A_c.map(computeGammaRRay)])
    gamma = gammaRRay.get(gammaRRay.argmax()).multiply(-1)

    prediction = prediction.add(u_A.multiply(gamma))
    coefficients = X.matrixSolve(prediction)

    coefficients = ee.Array(ee.List(coefficients.matrixTranspose().toList().get(0)).map(setZero))

    coeff_arr = ee.Array.cat([coeff_arr, coefficients], 1)

    return ee.Dictionary({'prediction':prediction, 'coeff_arr':coeff_arr})

In [15]:
prediction = ee.Array(ee.List.repeat([0], n))
coeff_arr = ee.Array(ee.List.repeat([0], m))
initial = ee.Dictionary({'prediction':prediction, 'coeff_arr':coeff_arr})

numIterations = ee.List.sequence(1, 10)
finalOutput = ee.Dictionary(numIterations.iterate(LARSregression, initial))
coeff_arr = finalOutput.getArray('coeff_arr')

In [17]:
coeff_arr = coeff_arr.getInfo()
pandas.DataFrame(coeff_arr)

,0,1,2,3,4,5,6,7,8,9,10
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
